## Introduction to QAMpy - Pilot DSP Module

Here we demonstrate our pilot-based DSP using QPSK pilots. It's integrated in the QAMpy package and processing can be done using either blind methods or pilot-based

It is build based on the following principle:    
Signal is divided into frames of a given length.
Each frame starts with a pilot-sequnence followed by the payload
Additional pilots are inserted into the payload for continous tracking
    
Frame strutured in the following way:
    
    Pilot Sequence: N pilot symbols long. 
        Used for: 
            Frame syncronization (alignment)
            Equalization and polarization de-multiplexing
            Frequency offset estimation
            
    Phase Pilots: Periodically inserted at a ratio of P (Ex. P=32 corresponds to 1 pilot and 31 payload symbols)
        Used for:
            Phase tracking
            
    Payload: Data kept in the pilot frame
        Our pilot-dsp uses no information from the data for processing and is  fully modulation format independent. 
                    

In [36]:
from qampy import signals, impairments, equalisation, phaserec, helpers
import numpy as np

# Simulation parameters
M=64
baud_rate = 32e9 # This is used to calculate normalized carrier impairments (frequency offset and phase nosie)
pil_ins_rat = 64
pil_seq_len = 1024
frame_len = 2**14
os = 2 # Oversampling (for rx processing)

# System impairments
snr = 20
lw = 10e3
fo = 100e6
omega = np.pi/5
dgd = 40e-12



C:\Users\mazurm\Desktop\PyCommunication\qampy\core\equalisation\equalisation.py:52: UserWarning: can not use cython training functions
  warnings.warn("can not use cython training functions")


ModuleNotFoundError: No module named 'qampy.core.equalisation.equaliser_numba'

## Demonstration using PM-64QAM Payload data
The signal object class contains functionality to generate a frame including both pilot symbols and payload. The pilot frame is just like any other object from the signal class and the same structure applies. 


Syntax:

    SignalWithPilots(M, frame_len, pilot_seq_len, pilot_ins_rat, nframes=1, pilot_scale=1, Mpilots=4,
                dataclass=SignalQAMGrayCoded, nmodes=1, dtype=np.complex128,  **kwargs)
    





In [12]:
## Genererate the signal with payload and pilots
pilot_signal = signals.SignalWithPilots(M,frame_len,pil_seq_len,pil_ins_rat,nframes=5,fb = baud_rate)

## Add all the impairments to the frame
pilot_signal = impairments.change_snr(pilot_signal,sel_snr)
pilot_signal = impairments.apply_phase_noise(pilot_signal, lw)
pilot_signal = impairments.add_carrier_offset(pilot_signal,fo)

# Resample to target 2 samples per symbol and add PMD
pilot_signal.resample(os)
pilot_signal = impairments.add_pmd(pilot_signal, omega, dgd)



NameError: name 'impairments' is not defined

In [8]:
### Now let's call for the pilot equalizer
import sim_pilot_txrx

pilots_out, payload_out, sync, taps, phase_trace, fo = 
sim_pilot_txrx.run_pilot_receiver(pilot_signal, pilot_signal.pilots, M = M, 
                                  pilot_seq_len = pil_seq_len, pilot_ins_ratio = pil_ins_rat)

SyntaxError: invalid syntax (<ipython-input-8-32567c60f853>, line 4)

In [27]:
### Now let's look at the output result
# here we use bokeh for plotting as it is much faster
from bokeh.io import output_notebook
from bokeh.plotting import figure, show
output_notebook()



Loading BokehJS ...

In [33]:
### Create a plotting function
def plot_constellation(E):
    fig = figure(title="QPSK signal constellation", output_backend="webgl")
    fig.scatter(E[0].real, E[0].imag, color='red', alpha=0.3, legend="X")
    fig.scatter(E[1].real, E[1].imag, color='blue', alpha=0.3, legend="Y")
    fig.xaxis[0].axis_label = "In-Phase"
    fig.yaxis[0].axis_label = "Quadrature"
    show(fig)

In [34]:
### Plot the output constellation
plot_constellation(payload_out[0]) # X-Pol
plot_constellation(payload_out[1]) # X-Pol


NameError: name 'payload_out' is not defined

In [31]:
## Calculate GMI-based performance metric
gmi = np.sum([pilot_signal.cal_gmi(payload_out[0]),pilot_signal.cal_gmi(payload_out[1])])


NameError: name 'np' is not defined